In [ ]:
import os
import sys
from dotenv import load_dotenv

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)
load_dotenv()

from decimal import Decimal
from core.data_sources import CLOBDataSource
from core.services.timescale_client import TimescaleClient

clob = CLOBDataSource()
ts_client = TimescaleClient(
    host=os.getenv("TIMESCALE_HOST", "localhost"),
    port=os.getenv("TIMESCALE_PORT", 5432),
    user=os.getenv("TIMESCALE_USER", "admin"),
    password=os.getenv("TIMESCALE_PASSWORD", "admin"),
    database="timescaledb"
)
await ts_client.connect()

In [ ]:
# Screener parameters
CONNECTOR_NAME = "binance_perpetual"
INTERVALS = ["1m"]

# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

# Config generation
TOTAL_AMOUNT = 300  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = 1  # Maximum number of executors per side
COOLDOWN_TIME = 0
LEVERAGE = 20  # Should be for each trading pair
TIME_LIMIT = 60 * 60 * 24
BOLLINGER_LENGTHS = [50, 100, 200]
BOLLINGER_STDS = [1.4, 1.8, 2.2]


In [ ]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET) \
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE)) \
    .get_all_trading_pairs()
trading_pairs_available = await ts_client.get_available_pairs()
trading_pairs_available = [pair[1] for pair in trading_pairs_available if pair[0] == CONNECTOR_NAME]

trading_pairs = [trading_pair for trading_pair in trading_pairs_available if trading_pair in trading_pairs]

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer


resolution = "1s"
optimizer = StrategyOptimizer(engine="postgres",
                              root_path=root_path,
                              resolution=resolution,
                              db_client=ts_client,
                              db_host=os.getenv("OPTUNA_HOST", "localhost"),
                              db_port=os.getenv("OPTUNA_LOCAL_PORT", 5433),
                              db_user=os.getenv("OPTUNA_USER", "admin"),
                              db_pass=os.getenv("OPTUNA_PASSWORD", "admin"))

In [ ]:
from research_notebooks.xtreet_bb.xtreet_bt import XtreetBacktesting
from research_notebooks.xtreet_bb.xtreet_config_gen_simple import XtreetConfigGenerator
import logging
import datetime

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

BACKTESTING_DAYS = 3

start_date = datetime.datetime.now() - datetime.timedelta(days=BACKTESTING_DAYS)
end_date = datetime.datetime.now()
logger.info("Generating config generator")

await optimizer._db_client.connect()
for trading_pair in trading_pairs[:1]:
    config_generator = XtreetConfigGenerator(start_date=start_date, end_date=end_date, backtester=XtreetBacktesting())
    config_generator.trading_pair = trading_pair
    candles = await optimizer._db_client.get_candles(CONNECTOR_NAME, trading_pair,
                                                     resolution, start_date.timestamp(), end_date.timestamp())
    start_time = candles.data["timestamp"].min()
    end_time = candles.data["timestamp"].max()
    config_generator.backtester.backtesting_data_provider.candles_feeds[
        f"{CONNECTOR_NAME}_{trading_pair}_{resolution}"] = candles.data
    config_generator.start = start_time
    config_generator.end = end_time

    await optimizer.optimize(study_name="xtreet_bb_v2",
                             config_generator=config_generator, n_trials=200)
